In [1]:
import numpy as np
import pandas as pd
import os
import sys
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

os.getcwd()
os.chdir('../..')
os.getcwd()

'C:\\Users\\p.schambach\\Desktop\\DSR\\drone_steering'

In [3]:
from app_local.module import DataEnsembler, GestureTransformer

de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/',is_frame_based=True)
de.load_data()
de.assemble_data()
de.display_information()

i: 0 	shape X: (603, 18, 16) 	shape y: (603,) 	count: 49
i: 1 	shape X: (519, 18, 16) 	shape y: (519,) 	count: 37
i: 2 	shape X: (498, 18, 16) 	shape y: (498,) 	count: 59
i: 3 	shape X: (650, 18, 16) 	shape y: (650,) 	count: 138
i: 4 	shape X: (508, 18, 16) 	shape y: (508,) 	count: 95
i: 5 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 90
i: 6 	shape X: (601, 18, 16) 	shape y: (601,) 	count: 107
i: 7 	shape X: (440, 18, 16) 	shape y: (440,) 	count: 83
i: 8 	shape X: (490, 18, 16) 	shape y: (490,) 	count: 70
i: 9 	shape X: (476, 18, 16) 	shape y: (476,) 	count: 74
i: 10 	shape X: (492, 18, 16) 	shape y: (492,) 	count: 85
i: 11 	shape X: (436, 18, 16) 	shape y: (436,) 	count: 59
i: 12 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 86
i: 13 	shape X: (551, 18, 16) 	shape y: (551,) 	count: 71
i: 14 	shape X: (550, 18, 16) 	shape y: (550,) 	count: 58
i: 15 	shape X: (549, 18, 16) 	shape y: (549,) 	count: 55
i: 16 	shape X: (541, 18, 16) 	shape y: (541,) 	count: 52
i: 17 	shape X: (535, 

In [4]:
X = de.X.copy()

In [ ]:
# number of 0-labels to use for traing
n = 750


# only select certain indeces to prevent too many 0-labeled instances
idx = []
for i in sorted(set(y)):
    idx.append(np.where(np.isclose(y,i))[0])

print("----- labels summary before --------------------------------")
for i in sorted(set(y)):
    print(i,len(idx[i]))
print("")
    
zero_idx = np.random.choice(idx[0], n, replace=False)
keep_idx = np.concatenate([zero_idx,idx[1],idx[2],idx[3],idx[4],idx[5],idx[6]])
keep_idx = sorted(keep_idx)

y = y[keep_idx]
X = X[keep_idx]
print("Shapes after:",y.shape, X.shape) 
print("")

idx = []
for i in sorted(set(y)):
    idx.append(np.where(np.isclose(y,i))[0])

print("----- labels summary after --------------------------------")
for i in sorted(set(y)):
    print(i,len(idx[i]))
print("")   

In [111]:
class DataResampler():
    
    def __init__(self):
        pass
        
    
    def fit(self, X, y):
        if X.shape[0] != y.shape[0]:
            raise AssertionError("First Dimension of Argument X and y have to be equal")
            
        self.orig_y = y.copy().astype("int32")
        self.orig_X = X.copy()
        
        self.X = self.orig_X.copy()
        self.y = self.orig_y.copy()
            
        self.labels = sorted(set(y.astype("int32")))
        
        maxl = max(self.labels)
        
        self.indices = {}
        for i in self.labels:
            self.indices[i] = np.where(np.isclose(y,i))[0]
            
            
    def restrict_zero_labels(self, n):
        y = self.orig_y.copy()
        
        n0 = len(self.indices[0])
        print("-------------------------------------")
        print("Current number of zero-labels:", n0)
        
        if n0 <= n:
            print("Target number of zero-labels is bigger than current number. No changes!")
        elif n < 0:
            print("Target number of zero-labels was negative. Treated as zero!")
        else:
            zero_idx = np.random.choice(self.indices[0], n, replace=False)
            other_labels_idx = np.concatenate([self.indices[i] for i in self.indices.keys() if i != 0])
            new_idx = sorted(np.concatenate([zero_idx, other_labels_idx]))
            
        
        print("New number of zero-labels:", n)
        
        self.X = self.X[new_idx,:,:]
        self.y = self.y[new_idx]

        return self.X, self.y 
    
    
    def upsample(self, n = None):
        if n is None:
            n = len(self.indices[0])
        elif n < 0:
            n = 0
        
        a = {} 
        b = {}
        idx = {}
            
        for i in self.indices.keys():
            a[i] = min(n, len(self.indices[i]))
            diff = n - len(self.indices[i])
            b[i] = max(0, diff)
            
            idx_a = np.random.choice(self.indices[i], a[i], replace = False)
            idx_b = np.random.choice(self.indices[i], b[i], replace = True)
            
            idx[i] = sorted(np.concatenate([idx_a, idx_b]))
            
        
        upsampled_idx = sorted(np.concatenate([idx[i] for i in idx]))
        
        self.X = self.orig_X[upsampled_idx,:,:]
        self.y = self.y[upsampled_idx]
        
        return self.X, self.y  
    
    def display_information(self):
        df = pd.DataFrame(columns=["abs_orig","pct_orig","abs_current","pct_current"],index = self.labels)
        
        n = len(self.y)
        
        for l in self.labels:
            nl = len(self.y[self.y == l])
            
            df.loc[l,"abs_orig"] = len(self.indices[l])
            df.loc[l,"pct_orig"] = np.round(100*len(self.indices[l])/len(self.orig_y),2)
            df.loc[l,"abs_current"] = nl
            df.loc[l,"pct_current"] = np.round(100*nl/n,2)
              
        return df

In [112]:
dr = DataResampler()
dr.fit(de.X,de.y)
X, y = dr.upsample()
print(X.shape, y.shape)
dr.display_information()

(109326, 18, 16) (109326,)


,abs_orig,pct_orig,abs_current,pct_current
0,15618,85.87,15618,14.29
1,356,1.96,15618,14.29
2,368,2.02,15618,14.29
3,575,3.16,15618,14.29
4,365,2.01,15618,14.29
5,449,2.47,15618,14.29
6,457,2.51,15618,14.29


In [94]:
de.y[[]]

array([0., 0.])